In [3]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.sparse import coo_matrix

In [4]:
#M = coo_matrix((3, 4), dtype=np.int8)#.toarray()
#M = np.ndarray((3,4))
M = np.zeros((3,4))
#M[0][15] = 5

In [5]:
type(M)

numpy.ndarray

In [22]:
#importing twitter data
dfs_t = pd.read_excel("C:/Users/menoci/Desktop/modelisation/Propagation_Virale/INTEGRATION_WEEK/twitter_dataset.xlsx", sheet_name= None, date_parser  = int)
t_post, t_user = dfs_t[list(dfs_t.keys())[1]] , dfs_t[list(dfs_t.keys())[2]]
t_post.set_index('id_user', drop = False , inplace = True)
t_user.set_index('id_user', drop = False, inplace = True)

dfs_i = pd.read_excel("C:/Users/menoci/Desktop/modelisation/Propagation_Virale/INTEGRATION_WEEK/instagram_dataset.xlsx", sheet_name= None, date_parser  = int)
i_post, i_user = dfs_i[list(dfs_i.keys())[1]] , dfs_i[list(dfs_i.keys())[2]]
i_post.set_index('id_user', drop = False,  inplace = True)
i_user.set_index('id_user',drop = False, inplace = True)


,id_user,nb_followers,nb_following,nb_posts,sex,id_followers
id_user,,,,,,
288877,288877,167,58,48,female,"[738818, 134147, 314454, 977416, 926730, 82740..."
140311,140311,67,72,8,female,"[380289, 341188, 775558, 998151, 246792, 17869..."
182096,182096,142,95,14,female,"[524806, 968200, 241324, 233490, 188948, 15054..."
208875,208875,249,99,150,male,"[776192, 164353, 989698, 134147, 305670, 65792..."
960092,960092,96,114,19,female,"[858624, 896013, 138779, 817185, 854563, 88580..."
205994,205994,74,114,7,female,"[901249, 368386, 200555, 386566, 974196, 66722..."
680388,680388,166,50,37,female,"[134147, 425989, 524806, 717831, 425933, 68814..."
314488,314488,112,248,67,male,"[377771, 357897, 898571, 820751, 873501, 77110..."
872813,872813,94,113,81,male,"[625666, 798315, 357897, 194575, 331281, 25499..."


In [7]:
#retweets_log = t_post[['id_user','id_twewt']]
t_post['infected_by'] = None
for index, row in t_post.iterrows():
    id_tweet_origin = row['id_tweet_origin']
    if id_tweet_origin != 0:
        infected_by = int(t_post[t_post['id_tweet'] == id_tweet_origin]['id_user'])
        t_post.loc[index,'infected_by'] = infected_by

                    

In [8]:
#tracking "fathers" on instragram
i_post['infected_by'] = None
for index, row in i_post.iterrows():
    id_post_origin = row['id_post_origin']
    if id_post_origin != 0:
        infected_by = int(i_post[i_post['id_post'] == id_post_origin]['id_user'])
        i_post.loc[index,'infected_by'] = infected_by

In [9]:
i_post

,id_user,id_post,date,time,half_day,location,views,reposts,likes,comments,id_post_origin,link_clicks,donation_tag,donation_val,house_buy,infected_by
id_user,,,,,,,,,,,,,,,,
672702,672702,638779430,2017-11-09,07:53,am,Avignon,2,18,0,0,0,True,False,0,False,None
474227,474227,953043456,2017-11-09,02:03,am,Toulon,36,11,16,1,0,True,False,0,False,None
587566,587566,650889385,2017-11-09,02:57,am,Paris,59,11,8,1,0,True,False,0,False,None
483543,483543,144265898,2017-11-09,11:43,am,Lyon,21,14,9,2,0,True,False,0,False,None
394103,394103,955542283,2017-11-09,04:53,pm,Toulon,16,3,2,0,638779430,False,False,0,False,672702
866330,866330,723527339,2017-11-09,11:16,pm,Paris,23,8,6,0,650889385,False,False,0,False,587566
537720,537720,662638612,2017-11-09,07:56,pm,Paris,31,10,7,1,144265898,False,False,0,False,483543
993508,993508,547874736,2017-11-09,08:15,pm,Strasbourg,39,8,3,2,638779430,False,False,0,False,672702
695586,695586,823986367,2017-11-09,03:38,pm,Avignon,7,8,3,0,650889385,False,False,0,False,587566


In [10]:
t_user.head()

,id_user,nb_followers,nb_following,nb_tweets,sex,birth_date,id_followers
id_user,,,,,,,
5662813,5662813,80,431,66,female,1985-11-11,"[2152321, 3954946, 3850627, 9664645, 9576070, ..."
6187946,6187946,189,171,75,male,1995-10-01,"[9904643, 7723529, 3763722, 8360535, 5897229, ..."
3122461,3122461,175,86,147,male,1982-03-20,"[2096133, 6092033, 8077327, 1885713, 7434770, ..."
1855435,1855435,217,35,121,female,1992-03-10,"[6206474, 4367884, 2206737, 3898900, 4445358, ..."
6561414,6561414,67,130,37,female,1991-12-03,"[1698176, 6092033, 5123435, 7344261, 4777606, ..."


In [17]:
#changing names of post table
t_post.rename(columns={'id_post':'id_tweet'}, inplace=True)
#t_post['id_post'] = t_post['id_tweet'] messes up the entire column

#changing names of users table
t_user.drop('birth_date',axis=1, inplace = True)
t_user.rename(columns={'id_tweet':'id_post'}, inplace=True)
i_user.rename(columns={'nb_posts':'nb_tweets'}, inplace=True)
i_post.rename(columns={'id_post_origin':'id_tweet_origin'}, inplace=True)


In [20]:
i_post

,id_user,id_post,date,time,half_day,location,views,reposts,likes,comments,id_tweet_origin,link_clicks,donation_tag,donation_val,house_buy
id_user,,,,,,,,,,,,,,,
672702,672702,638779430,2017-11-09,07:53,am,Avignon,2,18,0,0,0,True,False,0,False
474227,474227,953043456,2017-11-09,02:03,am,Toulon,36,11,16,1,0,True,False,0,False
587566,587566,650889385,2017-11-09,02:57,am,Paris,59,11,8,1,0,True,False,0,False
483543,483543,144265898,2017-11-09,11:43,am,Lyon,21,14,9,2,0,True,False,0,False
394103,394103,955542283,2017-11-09,04:53,pm,Toulon,16,3,2,0,638779430,False,False,0,False
866330,866330,723527339,2017-11-09,11:16,pm,Paris,23,8,6,0,650889385,False,False,0,False
537720,537720,662638612,2017-11-09,07:56,pm,Paris,31,10,7,1,144265898,False,False,0,False
993508,993508,547874736,2017-11-09,08:15,pm,Strasbourg,39,8,3,2,638779430,False,False,0,False
695586,695586,823986367,2017-11-09,03:38,pm,Avignon,7,8,3,0,650889385,False,False,0,False


In [18]:
#unifying posts

columns = ['id_user','id_post', 'infected_by', 'date','time','half_day']

df = pd.concat([i_post, t_post], axis=0)
df['time']=df["time"].astype('|S')

#unifying users
users = pd.concat([t_user,i_user], axis = 0)


C:\Users\menoci\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [21]:
t_post

,id_user,id_tweet,date,time,half_day,location,views,likes,comments,retweets,id_tweet_origin,link_clicks,donation_tag,donation_val,house_buy
id_user,,,,,,,,,,,,,,,
9834565,9834565,18271010,2017-11-09,09:41:00,am,Montreuil,62,24,2,16,0,False,False,0,False
1953787,1953787,55199327,2017-11-09,08:17:00,am,Reims,148,66,8,19,0,False,False,0,False
6013435,6013435,81242015,2017-11-09,05:18:00,am,Paris,78,23,0,10,0,False,False,0,False
3027418,3027418,68398591,2017-11-09,10:03:00,am,Villeurbanne,208,62,8,20,0,False,False,0,False
6027974,6027974,94580818,2017-11-09,08:10:00,am,Paris,23,9,0,18,0,False,False,0,False
3003097,3003097,48168379,2017-11-09,02:53:00,am,Marseille,26,9,1,20,0,False,False,0,False
9266081,9266081,50443972,2017-11-09,02:55:00,pm,Angers,93,23,2,9,48168379,False,False,0,False
5265603,5265603,53869774,2017-11-09,04:26:00,pm,Strasbourg,78,7,7,6,18271010,False,False,0,False
8820586,8820586,94306511,2017-11-09,07:23:00,pm,Tourcoing,35,15,0,3,81242015,False,False,0,False


In [19]:
df

,comments,date,donation_tag,donation_val,half_day,house_buy,id_post,id_tweet,id_tweet_origin,id_user,likes,link_clicks,location,reposts,retweets,time,views
id_user,,,,,,,,,,,,,,,,,
672702,0,2017-11-09,False,0,am,False,638779430.0,NaN,0,672702,0,True,Avignon,18.0,NaN,b'07:53',2
474227,1,2017-11-09,False,0,am,False,953043456.0,NaN,0,474227,16,True,Toulon,11.0,NaN,b'02:03',36
587566,1,2017-11-09,False,0,am,False,650889385.0,NaN,0,587566,8,True,Paris,11.0,NaN,b'02:57',59
483543,2,2017-11-09,False,0,am,False,144265898.0,NaN,0,483543,9,True,Lyon,14.0,NaN,b'11:43',21
394103,0,2017-11-09,False,0,pm,False,955542283.0,NaN,638779430,394103,2,False,Toulon,3.0,NaN,b'04:53',16
866330,0,2017-11-09,False,0,pm,False,723527339.0,NaN,650889385,866330,6,False,Paris,8.0,NaN,b'11:16',23
537720,1,2017-11-09,False,0,pm,False,662638612.0,NaN,144265898,537720,7,False,Paris,10.0,NaN,b'07:56',31
993508,2,2017-11-09,False,0,pm,False,547874736.0,NaN,638779430,993508,3,False,Strasbourg,8.0,NaN,b'08:15',39
695586,0,2017-11-09,False,0,pm,False,823986367.0,NaN,650889385,695586,3,False,Avignon,8.0,NaN,b'03:38',7


In [ ]:
#sorting values by date
df = df.sort_values(by=['date', 'half_day','time'])

In [ ]:
users

In [119]:
#construction of the graph
G = nx.DiGraph()
elist = []
for node, row in users.iterrows():
    #row['id_followers'] = row['id_followers'][1:-1].split(',')
    #adj_list.append((index,row['id_followers']))
    neighbors = row['id_followers'][1:-1].split(', ')
    #print(neighbors, type(neighbors))
    #print(neighbor)
    G.add_node(node)
    for n in neighbors:
        G.add_node(int(n))
        elist.append((node,int(n)))
G.add_edges_from(elist)

In [120]:
len(G)

7144

In [121]:
## I) Scan algorithm

In [122]:
#mapping between integers from 0 to max to vertices

In [123]:
mp = {}
i = 0
for index in list(df.index):
    mp[index] = i
    i += 1

In [124]:
len(mapping)

7144

In [138]:
def gamma(graph,v,u):
    return 1/graph.in_degree(u)

In [151]:
def scan(graph,L,lamb):
    '''Prend en argument:
    - graph, le graphe du réseau social
    - L, le log
    - lamb, un réel qui joue le rôle de seuil de troncature
    Renvoie un dictionnaire UC tel que UC[u][v] = Gamma_{v,u}^{V-S}'''
    UC = np.zeros((len(mapping)+1,len(mapping)+1))
    current_table = []
    node_list = L['id_user'] 
    parents = {}
    #for v in node_list: #Initialisation de UC
    #    UC[:][mp[v]] = 0
        
    for u in node_list:
        parents[mp[u]] = []
        #for v in node_list:
        #    UC[mp[v]][mp[u]] = 0
        if u in graph.nodes():
            for v in graph.neighbors(u):
                if v in current_table:
                    parents[mp[u]] += [v]
                
            for v in parents[mp[u]]:
                gamma2 = gamma(graph,v,u)
                if gamma2 >= lamb:
                    UC[mp[v]][mp[u]] += gamma2
                    for w in node_list:
                        if UC[mp[w]][mp[v]]*gamma2 >= lamb:
                            UC[mp[w]][mp[u]] += gamma2*UC[mp[w]][mp[v]]
        current_table += [u]
    return UC

In [152]:
## II) Algorithm Greedy with CELF

In [153]:
def computeMG(x,UC,SC):
    mg = 1
    for u in node_list:
        if UC[x][u] > 0:
            mg += UC[x][u]
    return mg*(1 - SC[x])

In [154]:
def update(x,UC,SC):
    for u in node_list:
        if UC[x][u] > 0:
            for v in node_list:
                if UC[v][x] > 0:
                    UC[v][u] -= UC[v][x] * UC[x][u]
            SC[u] += UC[x][u]*(1-SC[x])

In [155]:
def greedy(UC,k,L):
    SC = []
    S = []
    Q = []
    node_list = L['id_user']
    for u in node_list:
        mg = computeMG(u,UC,SC)
        it = 0
        Q = [(u,mg,it)] + Q
    while len(S) < k:
        (x,mg,it) = Q.pop()
        if it == len(S):
            S.append((x,mg,it))
            update((x,mg,it),UC,SC)
        else:
            mg = computeMG(x,UC,SC)
            it = len(S)
            Q = [(x,mg,it)] + Q
    return S

In [156]:
UC = scan(G,df,0.001)


In [157]:
UC

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.00609756,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [1]:
list(UC)

NameError: name 'UC' is not defined